<a href="https://colab.research.google.com/github/shirleyzz/cap5610_machineleanrning/blob/master/HW5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import tensorflow as tf
import keras
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense


# load train and test data
(train_images_original, train_labels_original), (test_images_original, test_labels_original) = mnist.load_data()


Using TensorFlow backend.


In [0]:
train_images_original.shape

(60000, 28, 28)

In [0]:

# preprocess data

train_images = train_images_original.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255

test_images = test_images_original.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels_original, num_classes=10) 
test_labels = to_categorical(test_labels_original, num_classes=10)

# set up layers
model = keras.Sequential([
    keras.layers.Dense(10, input_dim=784, activation=tf.nn.softmax)
])


# compile the model
model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

# train the model
epochs = 20
history = model.fit(train_images, 
                      train_labels, 
                      epochs=epochs,  
                      validation_data=(test_images, test_labels))